# Sudoku Board Image Generation

Generate 900×900px PNG images from Sudoku puzzle JSON data.

Key differences from KenKen:
- Bold lines for box boundaries (2×2 or 3×3 boxes)
- Numbers centered in cells (not top-left corner)
- Empty cells shown as blank (0 in puzzle = empty)
- No operators or targets

In [ ]:
import json
import os

with open("./puzzles/puzzles_dict.json", "r") as f:
    puzzles_ds = json.load(f)

print(f"Loaded puzzles: {list(puzzles_ds.keys())}")

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Load Font Data (TMNIST)

In [ ]:
# Try to load TMNIST data from KenKen folder
tmnist_path = "../KenKen/symbols/TMNIST_NotoSans.csv"

if os.path.exists(tmnist_path):
    df = pd.read_csv(tmnist_path)
    noto_sans = df[df['names'].str.contains('notosans', case=False, na=False)]
    noto_sans = noto_sans[noto_sans['names'].str.contains('NotoSans-Regular', case=False, na=False)]
    noto_sans = noto_sans.sort_values(by='labels')
    
    # Create digit images (0-9)
    digit_images = []
    for _, row in noto_sans.drop(columns=['names', 'labels']).iterrows():
        img = (255 - row.values.reshape(28, 28)) / 255.0
        digit_images.append(img)
    
    print(f"Loaded {len(digit_images)} digit images from TMNIST")
    USE_TMNIST = True
else:
    print("TMNIST not found, will use PIL text rendering")
    USE_TMNIST = False

## Board Generation Functions

In [ ]:
def make_sudoku_board(size):
    """
    Create a blank Sudoku board with grid lines.
    Bold lines for box boundaries.
    
    Args:
        size: 4 or 9
    
    Returns:
        900x900 numpy array (1.0 = white, 0.0 = black)
    """
    grid = np.ones((900, 900))
    box_size = 2 if size == 4 else 3
    cell_size = 900 // size
    
    # Draw outer border (bold)
    border_thickness = 8
    for i in range(border_thickness):
        grid[i, :] = 0
        grid[-i-1, :] = 0
        grid[:, i] = 0
        grid[:, -i-1] = 0
    
    # Draw internal grid lines
    for i in range(1, size):
        pos = i * cell_size
        
        # Bold lines for box boundaries
        if i % box_size == 0:
            thickness = 6
        else:
            thickness = 2
        
        for j in range(-thickness, thickness):
            if 0 <= pos + j < 900:
                grid[pos + j, :] = 0  # Horizontal line
                grid[:, pos + j] = 0  # Vertical line
    
    return grid

In [ ]:
def insert_digit_tmnist(grid, size, row, col, digit):
    """
    Insert a digit into a cell using TMNIST font images.
    Digit is centered in the cell.
    """
    if digit == 0:
        return grid  # Empty cell
    
    cell_size = 900 // size
    digit_size = cell_size * 2 // 3  # Digit takes 2/3 of cell
    
    # Get digit image
    digit_img = Image.fromarray(digit_images[digit])
    resized = digit_img.resize((digit_size, digit_size), resample=Image.LANCZOS)
    digit_arr = np.array(resized)
    
    # Calculate centered position
    cell_top = row * cell_size
    cell_left = col * cell_size
    offset = (cell_size - digit_size) // 2
    
    top = cell_top + offset
    left = cell_left + offset
    
    # Insert digit
    grid[top:top+digit_size, left:left+digit_size] = digit_arr
    
    return grid

In [ ]:
def insert_digit_pil(img, size, row, col, digit):
    """
    Insert a digit into a cell using PIL text rendering.
    Fallback when TMNIST is not available.
    """
    if digit == 0:
        return img
    
    cell_size = 900 // size
    draw = ImageDraw.Draw(img)
    
    # Try to use a nice font, fall back to default
    try:
        font_size = cell_size * 2 // 3
        font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", font_size)
    except:
        font = ImageFont.load_default()
    
    # Calculate centered position
    cell_center_x = col * cell_size + cell_size // 2
    cell_center_y = row * cell_size + cell_size // 2
    
    text = str(digit)
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    x = cell_center_x - text_width // 2
    y = cell_center_y - text_height // 2 - bbox[1]  # Adjust for baseline
    
    draw.text((x, y), text, fill=0, font=font)
    
    return img

In [ ]:
def make_board_full(size, puzzle):
    """
    Create a complete Sudoku board image with given clues.
    
    Args:
        size: Grid size (4 or 9)
        puzzle: 2D list where 0 = empty cell
    
    Returns:
        900x900 numpy array or PIL Image
    """
    if USE_TMNIST:
        grid = make_sudoku_board(size)
        for row in range(size):
            for col in range(size):
                digit = puzzle[row][col]
                if digit != 0:
                    grid = insert_digit_tmnist(grid, size, row, col, digit)
        return grid
    else:
        # Use PIL rendering
        grid = make_sudoku_board(size)
        img = Image.fromarray((grid * 255).astype(np.uint8), mode='L')
        for row in range(size):
            for col in range(size):
                digit = puzzle[row][col]
                if digit != 0:
                    img = insert_digit_pil(img, size, row, col, digit)
        return np.array(img) / 255.0

In [ ]:
def make_and_save(size, puzzle, idx):
    """
    Generate and save a Sudoku board image.
    """
    board = make_board_full(size, puzzle)
    array_uint8 = (board * 255).astype(np.uint8)
    image = Image.fromarray(array_uint8, mode='L')
    
    output_dir = './board_images'
    os.makedirs(output_dir, exist_ok=True)
    
    filename = f"{output_dir}/board{size}_{idx}.png"
    image.save(filename)
    return filename

## Demo: Generate Single Board

In [ ]:
# Demo with first puzzle from each size
for size_str in puzzles_ds.keys():
    size = int(size_str)
    if len(puzzles_ds[size_str]) > 0:
        puzzle = puzzles_ds[size_str][0]['puzzle']
        board = make_board_full(size, puzzle)
        
        plt.figure(figsize=(6, 6))
        plt.imshow(board, cmap='gray')
        plt.title(f"{size}×{size} Sudoku Puzzle")
        plt.axis('off')
        plt.show()

## Generate All Board Images

In [ ]:
# Generate images for all puzzles
total_generated = 0

for size_str in puzzles_ds.keys():
    size = int(size_str)
    puzzles = puzzles_ds[size_str]
    
    print(f"Generating {len(puzzles)} images for {size}×{size} puzzles...")
    
    for i, puzzle_data in enumerate(puzzles):
        puzzle = puzzle_data['puzzle']
        make_and_save(size, puzzle, i)
        
        if (i + 1) % 25 == 0:
            print(f"  Generated {i + 1}/{len(puzzles)}")
    
    total_generated += len(puzzles)

print(f"\nTotal images generated: {total_generated}")

## Verify Generated Images

In [ ]:
# List generated images
import glob

images = glob.glob('./board_images/*.png')
print(f"Total images in board_images: {len(images)}")

# Count by size
for size in [4, 9]:
    count = len([f for f in images if f'board{size}_' in f])
    print(f"  {size}×{size}: {count} images")

In [ ]:
# Display a few random examples
import random

sample_images = random.sample(images, min(4, len(images)))

fig, axes = plt.subplots(1, len(sample_images), figsize=(4*len(sample_images), 4))
if len(sample_images) == 1:
    axes = [axes]

for ax, img_path in zip(axes, sample_images):
    img = Image.open(img_path)
    ax.imshow(img, cmap='gray')
    ax.set_title(os.path.basename(img_path))
    ax.axis('off')

plt.tight_layout()
plt.show()